This notebook experiments with building out a fully customized TerriaJS catalog from a search result of ScienceBase Items, transforming sbJSON into the json catalog format used by TerriaJS. Resulting catalog JSON files are output to the local directory, deployed to GitHub, and then callable into an out of the box TerriaJS mapper.

In [2]:
%%bash

pip install sciencebasepy

In [0]:
import time
import os
import sciencebasepy
from getpass import getpass
import bs4
import requests
import json
import base64
import subprocess
import datetime
import getpass
import random
import string

from IPython.display import display

In [23]:
# Get what we want to work with using a ScienceBase /items/ query
#sbURL = "https://www.sciencebase.gov/catalog/items?parentId=585400f6e4b0e2663625f36f&format=json&fields=title,body,distributionLinks&loadInstances=true"
sbURL="https://www.sciencebase.gov/catalog/items?max=200&parentId=5644f3c1e4b0aafbcd0188f1&format=json&fields=title,body,distributionLinks,facets&&loadInstances=true"
sbJSON = requests.get(sbURL).json()
display (sbJSON)

{'items': [{'body': 'A subset of files from within processing region 4 of the NHDPlus Version 1. Although reorganized, the files within the attachments are unaltered from the NHDPlus Version 1 (see acquisition date listed within this metadata). This item links to python code used to generate the item.',
   'distributionLinks': [],
   'id': '5a3d5690e4b0d05ee8b8e63f',
   'link': {'rel': 'self',
    'url': 'https://www.sciencebase.gov/catalog/item/5a3d5690e4b0d05ee8b8e63f'},
   'relatedItems': {'link': {'rel': 'related',
     'url': 'https://www.sciencebase.gov/catalog/itemLinks?itemId=5a3d5690e4b0d05ee8b8e63f'}},
   'title': 'NHDPlusV1 GreatLakes; cached files for use in the Biogeographic Information System'},
  {'body': 'A subset of files from within processing region 6 of the NHDPlus Version 1. Although reorganized, the files within the attachments are unaltered from the NHDPlus Version 1 (see acquisition date listed within this metadata). This item links to python code used to genera

In [25]:
theCatalog = {}
theCatalog["corsDomains"] = ["sciencebase.gov"]
#theCatalog["baseMapName"] = ""
theCatalog["initialViewerMode"] = "2d"
theCatalog["services"] = []

theCatalog["homeCamera"] = {}
theCatalog["homeCamera"]["west"] = -175
theCatalog["homeCamera"]["east"] = -66
theCatalog["homeCamera"]["north"] = 72
theCatalog["homeCamera"]["south"] = 18

theCatalog["catalog"] = []

padusGroup = {}
padusGroup["name"] = "Wetland Observatory Data"
padusGroup["type"] = "group"
padusGroup["isOpen"] = "true"

padusGroup["items"] = []

for idx, item in enumerate(sbJSON["items"]):
    if 'body' not in item:
        continue
    if 'facets' not in item:
        continue
    thisItem = {}
    thisItem["id"] = item["id"]
    thisItem["name"] = item["title"]
    thisItem["description"] = item["body"]
    serviceCount = 0

    for link in item["facets"]:
        if 'type' not in link:
          continue
        if link["type"] == "WMS":
            serviceCount = serviceCount + 1
            if serviceCount > 1:
                break
            thisItem["type"] = "wms"
            thisItem["url"] = link["serviceMapUrl"]
            thisItem["layers"] = link["layers"]
    if serviceCount == 0:
        continue
    padusGroup["items"].append(thisItem.copy())

theCatalog["catalog"].append(padusGroup.copy())

display (theCatalog)

f = open('catalog.json', 'wt', encoding='utf-8')
#f.write(json.dumps(theCatalog).replace(': "true"', ': true'))

{'baseMapName': 'Bing Maps Roads',
 'catalog': [{'isOpen': 'true',
   'items': [{'description': 'The Average Bloom PRISM is a synthetic measure of these early season events in plants, based on recent temperature conditions. This model allows us to track the progression of spring onset across the country.<br>\n<br>\nPRISM is a set of monthly, yearly, and single-event gridded data products of mean temperature and precipitation, max/min temperatures, and dewpoints, primarily for the United States. In-situ point measurements are ingested into the PRISM (Parameter elevation Regression on Independent Slopes Model) statistical mapping system. The PRISM products use a weighted regression scheme to account for complex climate regimes associated with orography, rain shadows, temperature inversions, slope aspect, coastal proximity, and other factors. Climatologies (normals) are available at 30-arcsec (800 meters) and monthly data are available at 2.5 arcmin (4 km) resolution. PRISM is the USDA\'s

5729